In [1]:
import findspark
findspark.init('/home/adam/EPFL_courses/spark-1.6.3-bin-hadoop2.6')

In [2]:
!python --version

Python 3.4.5 :: Anaconda 4.3.1 (64-bit)


In [3]:
import pandas as pd
from scipy import stats
import seaborn as sns
import numpy as np
import json
import re
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SQLContext
import json 
import gzip
from pyspark.sql.functions import to_date, unix_timestamp, from_unixtime #, to_timestamp, 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def parse(path): 
    g = gzip.open(path, 'r') 
    for l in g:
        yield json.loads(l) 

In [ ]:
number_of_reviews = 0
g = gzip.open("/home/adam/Downloads/reviews_Books_5.json.gz")
for l in g:
    number_of_reviews = number_of_reviews + 1
print(number_of_reviews)
g.close()

In [4]:
#SparkContext.setSystemProperty('spark.executor.memory', '2g')
sc = SparkContext()
sqlContext = SQLContext(sc)

In [ ]:
SparkContext.setSystemProperty('spark.driver.maxResultSize', '4g')

In [ ]:
SparkContext.setSystemProperty('spark.executor.extrajavaoptions', '-Xmx1024m')

In [ ]:
SparkContext.setSystemProperty('spark.sql.autoBroadcastJoinThreshold', '-1')

In [ ]:
sc.version

In [5]:
text_file = sc.textFile("/media/adam/B236CB1D36CAE209/Studia/ADA/reviews_Books_5.json")
df = sqlContext.read.json(text_file)

In [6]:
metadata = sc.textFile("/media/adam/B236CB1D36CAE209/Studia/ADA/meta_Books.json")
metadata_df = sqlContext.read.json(metadata)

In [7]:
sqlContext.registerDataFrameAsTable(metadata_df, "metadata")

sqlContext.registerDataFrameAsTable(df, "dataset")

In [ ]:
df.alias('a').join(metadata_df.alias('b'),col('b.asin') == col('a.asin')).take(5)

In [ ]:
text_file.getNumPartitions()

In [ ]:
df.printSchema()

In [ ]:
df = df.withColumn('unixReviewTime', from_unixtime(df['unixReviewTime']))
df = df.withColumn('reviewTime', to_date(df['unixReviewTime']))
df = df.withColumn('unixReviewTime', df['unixReviewTime'].cast('timestamp'))
df.printSchema()

In [ ]:
df.select("reviewTime", 'reviewText', 'unixReviewTime').take(2)

In [ ]:
aTuple = (0, 0)
a = df.select("reviewTime", 'reviewText').rdd.map(lambda row: (row.reviewTime, len(row.reviewText)))
a = a.aggregateByKey(aTuple, lambda a,b: (a[0] + b, a[1] + 1), lambda a,b: (a[0] + b[0], a[1] + b[1]))
a = a.mapValues(lambda v: v[0]/v[1])
a = a.collect()

In [ ]:
len(a)

In [ ]:
text_file.count()

In [ ]:
avg_len = pd.DataFrame(a, columns=['Date', 'Avg_length'])
avg_len['Date'] = pd.to_datetime(avg_len['Date'])
avg_len.set_index('Date', inplace=True)
avg_len.sort_index(inplace=True)
avg_len.head()

In [ ]:
avg_len.to_csv("avg_length_review_by_day.csv")

In [ ]:
monthly_data = avg_len.groupby(avg_len.index.to_period('M')).mean()
monthly_data.plot(figsize=(20,10))

In [ ]:
monthly_data['2012':'2013'].plot(figsize=(20,10))

In [ ]:
monthly_data.to_csv("avg_length_review_by_month.csv")

In [ ]:
tmp = df.rdd.filter(lambda row: row.unixReviewTime > pd.to_datetime('2012-05')
                     and row.unixReviewTime < pd.to_datetime('2013'))
tmp.take(5)

In [ ]:
number_of_reviews = df.rdd.map(lambda row: (row.reviewTime, 1)).reduceByKey(lambda a, b: a+b).collect()

In [ ]:
rev_num = pd.DataFrame(number_of_reviews, columns=['Date', 'Number of reviews'])
rev_num['Date'] = pd.to_datetime(rev_num['Date'])
rev_num.set_index('Date', inplace=True)
rev_num.sort_index(inplace=True)
rev_num.head()

In [ ]:
rev_num.to_csv("number_of_reviews_per_day.csv")

In [ ]:
monthly_data_reviews = rev_num.groupby(rev_num.index.to_period('M')).sum()
monthly_data_reviews.plot(figsize=(20,10))

In [ ]:
monthly_data_reviews.to_csv("number_of_reviews_per_month.csv")

In [ ]:
number_of_reviews_per_book = df.rdd.map(lambda row: (row.asin, 1)).reduceByKey(lambda a, b: a+b).collect()

In [8]:
test_df = df.rdd.map(lambda row: (row.asin, 1)).reduceByKey(lambda a, b: a+b).sortBy(lambda wc: -wc[1]).take(30)

In [9]:
test_df = sqlContext.createDataFrame(test_df, ['asin', 'rew_num'])

In [10]:
test_df.take(5)

[Row(asin='030758836X', rew_num=7440),
 Row(asin='0439023483', rew_num=6717),
 Row(asin='0375831002', rew_num=4864),
 Row(asin='038536315X', rew_num=4604),
 Row(asin='0439023513', rew_num=4440)]

In [12]:
sqlContext.registerDataFrameAsTable(test_df, "test")

In [ ]:
sqlContext.sql("select asin, title from metadata")

In [11]:
test_df.join(sqlContext.sql("select asin, title from metadata")).take(5)

[Row(asin='030758836X', rew_num=7440, asin='0001048791', title='The Crucible: Performed by Stuart Pankin, Jerome Dempsey &amp; Cast'),
 Row(asin='030758836X', rew_num=7440, asin='0001048775', title='Measure for Measure: Complete &amp; Unabridged'),
 Row(asin='030758836X', rew_num=7440, asin='0001048236', title='The Sherlock Holmes Audio Collection'),
 Row(asin='030758836X', rew_num=7440, asin='0000401048', title="The rogue of publishers' row;: Confessions of a publisher (A Banner Book)"),
 Row(asin='030758836X', rew_num=7440, asin='0001019880', title="Classic Soul Winner's New Testament Bible")]

In [15]:
sqlContext.sql("select t.asin, m.title from metadata m join test t on m.asin=t.asin").collect()

[Row(asin='1940026016', title='The Atlantis Gene: A Thriller (The Origin Mystery, Book 1)'),
 Row(asin='0439023483', title='The Hunger Games (The Hunger Games, Book 1)'),
 Row(asin='0316206849', title="The Cuckoo's Calling"),
 Row(asin='1469984202', title='Wool - Omnibus Edition'),
 Row(asin='0141039280', title='The Help'),
 Row(asin='0385537859', title='Inferno'),
 Row(asin='038536315X', title='Sycamore Row'),
 Row(asin='0312853238', title="Ender's Game (The Ender Quintet)"),
 Row(asin='0007444117', title='Allegiant (Divergent, #3)'),
 Row(asin='0316055433', title='The Goldfinch: A Novel (Pulitzer Prize for Fiction)'),
 Row(asin='0007386648', title='Unbroken'),
 Row(asin='030758836X', title='Gone Girl'),
 Row(asin='0345803485', title='Fifty Shades of Grey: Book One of the Fifty Shades Trilogy'),
 Row(asin='0425263924', title='Entwined with You (Crossfire, Book 3)'),
 Row(asin='0857521012', title='The Light Between Oceans'),
 Row(asin='0061950726', title='Orphan Train: A Novel'),
 Row(

In [ ]:
sqlContext.sql("select t.asin, m.title from metadata m join test t limit 10").collect()

In [ ]:
test_df.join(metadata_df.select("asin", "title")).collect()

In [ ]:
len(number_of_reviews_per_book)

In [ ]:
df_number_of_reviews_per_book = pd.DataFrame(number_of_reviews_per_book, columns=['Book_id', 'Number of reviews'])

In [ ]:
df_number_of_reviews_per_book.sort_values('Number of reviews', ascending=False, inplace = True)
df_number_of_reviews_per_book.to_csv("number_of_reviews_per_book.csv", index=False)
df_number_of_reviews_per_book.head(10)

In [ ]:
plt.hist(np.array(df_number_of_reviews_per_book['Number of reviews'].values), bins = 1000)
plt.show()

In [ ]:
stats.describe(df_number_of_reviews_per_book['Number of reviews'].values)

In [ ]:
sns.boxplot(df_number_of_reviews_per_book['Number of reviews'].values)

In [ ]:
most_reviewed_books_id_top10 = df_number_of_reviews_per_book[:30]
tmp = df.rdd.filter(lambda row: row.asin in list(most_reviewed_books_id_top10.Book_id)).map(lambda row: (row.asin, row.reviewTime)).collect()

In [ ]:
temp = sqlContext.sql("select asin, reviewTime from dataset where asin in " + 
               str(tuple(most_reviewed_books_id_top10.Book_id)))

In [ ]:
most_reviewed_books_top20_df = pd.DataFrame(tmp, columns=['asin', 'reviewTime'])
most_reviewed_books_top20_df['reviewTime'] = pd.to_datetime(most_reviewed_books_top20_df['reviewTime'])

In [ ]:
most_reviewed_books_top20_df['Number_of_reviews'] = 1
most_reviewed_books_top20_df['Year-month'] = most_reviewed_books_top20_df['reviewTime'].dt.to_period('M')
most_reviewed_books_top20_df.head()

In [ ]:
most_reviewed_books_top20_df.groupby(['asin', 'reviewTime']).sum().to_csv("number_of_reviews_per_day_top30_books.csv")
m_rev_books_by_month = most_reviewed_books_top20_df.groupby(['asin', 'Year-month'], as_index=True).sum()
m_rev_books_by_month.to_csv("number_of_reviews_per_month_top30_books.csv")

In [ ]:
m_rev_books_by_month.unstack(level=0).to_csv("number_of_reviews_per_month_top30_books_UNSTACKED.csv")
m_rev_books_by_month.unstack(level=0).plot(figsize = (20,10))

In [ ]:
fun = np.vectorize(lambda x: x.to_timestamp())
m_rev_books_by_month['Timestamp'] = fun(m_rev_books_by_month['Year-month'].values)

In [ ]:
m_rev_books_by_month.set_index('asin')

In [ ]:
list(most_reviewed_books_id_top10.Book_id)

In [ ]:
aTuple = (0, 0)
b = df.select('asin', 'reviewText').rdd.map(lambda row: (row.asin, len(row.reviewText)))
b = b.aggregateByKey(aTuple, lambda a,b: (a[0] + b, a[1] + 1), lambda a,b: (a[0] + b[0], a[1] + b[1]))
b = b.mapValues(lambda v: (v[0]/v[1], v[1]))
b = b.collect()

In [ ]:
b = [(k, v1, v2) for k, (v1, v2) in b]

In [ ]:
avg_len_review_per_book = pd.DataFrame(b, columns=['Book_id', 'Avg_len', 'number_of_reviews'])
avg_len_review_per_book.sort_values(['Avg_len', 'number_of_reviews'], ascending=False, inplace=True)
avg_len_review_per_book.head(10)

In [ ]:
avg_len_review_per_book.to_csv("avg_length_and_number_of_reviews_per_book.csv", index=False)

In [ ]:
sc.stop()